# Direct Inversion of Iterative Subspace

In [1]:
import numpy as np
import scipy.linalg as spla
import psi4
import time

## Useful Resources
- [P. Pulay. Chem. Phys. Lett. 73, 393-398 (1980)](https://www.sciencedirect.com/science/article/pii/0009261480803964)
- [DIIS by C. David Sherril](http://vergil.chemistry.gatech.edu/notes/diis/diis.pdf)
- [DePrince Research Group DIIS Tutorial](https://www.chem.fsu.edu/~deprince/programming_projects/diis/)
- [Psi4Numpy DIIS Tutorial](https://github.com/psi4/psi4numpy/blob/master/Tutorials/03_Hartree-Fock/3b_rhf-diis.ipynb)
- [DIIS by MolSSI-Education](https://github.com/MolSSI-Education/QM_2017_SSS_Team8/blob/master/Tutorial_PDFs/02_SCF_DIIS.pdf)

## Introduction
Iterative methods are usually used in order to solve systems of linear equations. These methods can suffer from numerous convergence issues such as slow convergence and high computational cost. Today we are going to work with DIIS to accelerate our convergence. DIIS stands for Direct Inversion of Iterative Subspace and is commonly used to aid in the convergence of SCF wavefunctions. Today we will build off of our previous example of a simple RHF.

## General Theory
During the iterative solution we generate a set of trial vectors $p^{i}$ that are converging to the true solution $p^{f}$. This allows for us to form a set of residual vectors
$$
\Delta \mathbf{p} = \mathbf{p}^{i+1} - \mathbf{p}^{i}
$$

DIIS assumes that the true solution can be approximated as a linear combination of the previous trial vector guesses, 
$$\mathbf{p} = \sum_{i} c_{i} \mathbf{p}^{i}$$


The coefficients $c_{i}$ can be obtained by requiring the residual vector to be a least-squares approximate to the zero vector 

$$\Delta \mathbf{p} = \sum_{i} c_{i} \Delta \mathbf{p}^{i}$$


constrained by,

$$\sum_{i} c_{i} =1$$


This allows for us to to represent each trial function $p^{i}$  as the true solution plus an error vector.  
$$\mathbf{p} = \sum_{i} c_{i} (\mathbf{p}^{f} + \mathbf{e}^{i}) = \mathbf{p}^{f} \sum_{i} c_{i} + \sum_{i} c_{i} \mathbf{e}^{i}$$

Convergence will result in minimizing the error which in turn causes the second term above to vanish. For our DIIS solution $\mathbf{p}$ to be equal to the true solution $\mathbf{p}^{f}$, we must have $\sum_{i} c_{i} =1$.

Need to minimize the norm of the residual vector subject to the constraint
$$ \left \langle \Delta \mathbf{p} | \Delta \mathbf{p} \right \rangle = \sum_{ij}  c_{i}^{\ast} c_{j} \left \langle \Delta \mathbf{p}^{i} | \Delta \mathbf{p}^{j} \right \rangle $$

We can minimize using a Lagrange multiplier
$$ \cal L = c^{\dagger} \mathbf{B} c - \lambda (1 - \sum_{i}  c_{i})$$

where B is the residual vector overlap.
$$ B_{ij}=\left \langle \Delta \mathbf{p}^{i} | \Delta \mathbf{p}^{j} \right \rangle $$

This allows for us to minimize $\cal L$ with respect to a coeff $c_{k}$
$$\frac{\partial \cal L }{\partial c_{k}}=0 = \sum_{j}  c_{j} B_{kj} +  \sum_{i}  c_{i} B_{ik} - \lambda = 2 \sum_{i}  c_{i} B_{ik} - \lambda$$

We can represent this with the matrix below

$$
\begin{bmatrix}
B_{11} & B_{12} & \cdots  & B_{1m} & -1 & \\ 
B_{21} & B_{22} & \cdots & B_{2m} & -1 & \\ 
\vdots  & \vdots  & \ddots   & \vdots  & \vdots  & \\ 
B_{m1} & B_{m2} & \cdots & B_{mm} & -1 & \\ 
-1 & -1 & \cdots & -1 & 0 & 
\end{bmatrix} 
\begin{bmatrix}
c_{1} & \\ 
c_{2} &   \\ 
\vdots  &   \\ 
c_{m} &   \\ 
\lambda &  
\end{bmatrix} 
=
\begin{bmatrix}
0 & \\ 
0 &   \\ 
\vdots &   \\ 
0 &   \\ 
-1 &  
\end{bmatrix} 
$$

## Imports

## Load Molecule

In [2]:
# Define molecule
mol = psi4.geometry("""
O 0.0000000 0.0000000 0.0000000
H 0.7569685 0.0000000 -0.5858752
H -0.7569685 0.0000000 -0.5858752
symmetry c1
""")
psi4.set_options({'basis': 'sto-3g'})
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('BASIS'))
mints = psi4.core.MintsHelper(wfn.basisset())

# Get number of electrons
num_elec_alpha =  wfn.nalpha()
num_elec_beta =  wfn.nbeta()
num_elec = num_elec_alpha + num_elec_beta

# Get nuclear repulsion energy
E_nuc =  mol.nuclear_repulsion_energy()

## Calculate Molecular Integrals

In [3]:
# Calculate overlap integrals
S = np.asarray(mints.ao_overlap())

# Calculate kinetic energy integrals
T = np.asarray(mints.ao_kinetic())

# Calculate nuclear attraction integrals
V = np.asarray(mints.ao_potential())

# Form core Hamiltonian
H = T + V

# Calculate two electron integrals
eri = np.asarray(mints.ao_eri())

# Get number of atomic orbitals
num_ao = np.shape(S)[0]

print(np.shape(eri))

(7, 7, 7, 7)


## Core Guess

In [4]:
# AO orthogonalization matrix 
A = spla.fractional_matrix_power(S, -0.5)

# Solve the generalized eigenvalue problem
E_orbitals, C = spla.eigh(H,S)

# Compute initial density matrix
D = np.zeros((num_ao,num_ao))
for i in range(num_ao):
    for j in range(num_ao):
        for k in range(num_elec_alpha):
            D[i,j] +=  C[i,k] * C[j,k]

## DIIS Function

### Steps in DIIS Function
1. Build B matrix
2. Solve the Pulay equation
3. Build the DIIS Fock matrix

In [5]:
def diis(F_list, diis_res):
    # Build B matrix 
    dim_B = len(F_list) + 1
    B = np.empty((dim_B, dim_B))
    B[-1, :] = -1
    B[:, -1] = -1
    B[-1, -1] = 0
    for i in range(len(F_list)):
        for j in range(len(F_list)):
            B[i, j] = np.einsum('ij,ij->', diis_res[i], diis_res[j])
    
    # Right hand side of Pulay eqn
    right = np.zeros(dim_B)
    right[-1] = -1

    # Solve Pulay for coeffs
    cn = np.linalg.solve(B, right)
    
    # Build DIIS Fock
    F_diis = np.zeros_like(F_list[0])
    for x in range(cn.shape[0] - 1):
        F_diis += cn[x] * F_list[x]
        
    return F_diis

## Variables, Criteria, and Organization

In [6]:
# 2 helper functions for printing during SCF
def print_start_iterations():
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("Iter", "Time(s)", "DIIS RMS", "delta E", "E_elec")))
    print("{:^79}".format("{:>4}  {:>11}  {:>11}  {:>11}  {:>11}".format("****", "*******", "*******", "*******", "******")))
def print_iteration(iteration_num, iteration_start_time, iteration_end_time, diis_rms, iteration_E_diff, E_elec):
    print("{:^79}".format("{:>4d}  {:>11f}  {:>.5E}  {:>.5E}  {:>11f}".format(iteration_num, iteration_end_time - iteration_start_time, diis_rms, iteration_E_diff, E_elec)))

# Set stopping criteria
iteration_max = 100
convergence_E = 1e-9
convergence_DIIS = 1e-5

# Loop variables
iteration_num = 0
E_total = 0
E_elec = 0.0
iteration_E_diff = 0.0
iteration_rmsc_dm = 0.0
converged = False
exceeded_iterations = False

## DIIS SCF Iteration
Our trial vector will be the Fock matrix with the error vector being the orthonormalized orbital gradient.

$$ r_{\mu \upsilon} = (\mathbf{A^{T}}(\mathbf{FDS} - \mathbf{SDF}) \mathbf{A})_{\mu \upsilon} $$


### Call DIIS in SCF Iteration
1. Build DIIS Residual (error vector) that will be used to make the B matrix
2. Store trial and residual vectors
3. Call DIIS to start after the first iteration
4. Compute the next guess with the DIIS Fock matrix

In [7]:
# Trial & Residual vector lists
F_list = []
DIIS_resid = []

print("{:^79}".format('=====> Starting SCF Iterations <=====\n'))
print_start_iterations()
while (not converged and not exceeded_iterations):
    # Store last iteration and increment counters
    iteration_start_time = time.time()
    iteration_num += 1
    E_elec_last = E_elec
    D_last = np.copy(D)
    
    # Form G matrix
    G = np.zeros((num_ao,num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_ao):
                for l in range(num_ao):
                    G[i,j] += D[k,l] * ((2.0*(eri[i,j,k,l])) - (eri[i,k,j,l]))
    
    # Build fock matrix
    F  = H + G
    
    # Calculate electronic energy
    E_elec = np.sum(np.multiply(D , (H +  F)))
    # E_elec = np.einsum("pq,pq->", (H + F), D)
    
    # Calculate energy change of iteration
    iteration_E_diff = np.abs(E_elec - E_elec_last)
    
    #=======> Start of DIIS stuff <=========
    # Build the DIIS AO gradient
    diis_r = A.T @ (F @ D @ S - S @ D @ F) @ A
    
    # DIIS RMS
    diis_rms = np.mean(diis_r**2)**0.5

    # Append lists
    F_list.append(F)
    DIIS_resid.append(diis_r)

    if iteration_num >=2:
        # preform DIIS to get Fock Matrix
        F = diis(F_list, DIIS_resid)
    
    # Compute new guess with F DIIS
    E_orbitals, C = spla.eigh(F,S)
    D = np.zeros((num_ao,num_ao))
    for i in range(num_ao):
        for j in range(num_ao):
            for k in range(num_elec_alpha):
                D[i,j] +=  C[i,k] * C[j,k]
    
    #=======> End of DIIS stuff <=========
    
    iteration_end_time = time.time()
    print_iteration(iteration_num, iteration_start_time, iteration_end_time, 
                    diis_rms, iteration_E_diff, E_elec)
    
    if(np.abs(iteration_E_diff) < convergence_E and diis_rms < convergence_DIIS): 
        converged = True
        print('\n',"{:^79}".format('=====> SCF Converged <=====\n'))
        # calculate total energy
        E_total = E_elec + E_nuc
        print("{:^79}".format("Total Energy : {:>11f}".format(E_total)))
    
    if(iteration_num == iteration_max):
        exceeded_iterations = True
        print("{:^79}".format('=====> SCF Exceded Max Iterations <=====\n'))

                    =====> Starting SCF Iterations <=====
                     
           Iter      Time(s)     DIIS RMS      delta E       E_elec            
           ****      *******      *******      *******       ******            
              1     0.003409  1.49013E-01  8.24274E+01   -82.427351            
              2     0.003355  2.03413E-02  1.71341E+00   -84.140761            
              3     0.003271  4.09380E-03  1.61315E-02   -84.156893            
              4     0.004129  7.23932E-04  8.62038E-04   -84.157755            
              5     0.003383  1.66275E-04  3.59959E-05   -84.157791            
              6     0.004169  1.73151E-05  1.88455E-06   -84.157793            
              7     0.003132  4.14841E-08  1.88806E-08   -84.157793            
              8     0.003428  7.00273E-10  9.94760E-14   -84.157793            

                          =====> SCF Converged <=====
                          
                          Total Energy

## References
1. P. Pulay. Chem. Phys. Lett. 73, 393-398 (1980)
2. C. David Sherrill. "Some comments on accellerating convergence of iterative sequences using direct inversion of the iterative subspace (DIIS)". http://vergil.chemistry.gatech.edu/notes/diis/diis.pdf. (1998)